# Investigating reports - BHP (coal)

In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')

#from textracting.s3_downloading import download_reports

In [30]:
df = pd.read_excel('QDEX_metada_export.xlsx')

In [27]:
df = df.loc[df.RSTATUS != 'C']

In [32]:
df.shape

(100624, 12)

In [28]:
# find number of reports of each type
#types = ['6MTH', 'ANNUAL', 'FINAL', 'FINREQ', 'HFACR', 'INIFIN', 'MINAW', 'PETFLD', 'PETNAW', 'STSURV', 'WELCOM', 'WELPRO', 'WELTST']
types = ['CDIFIN', 'CUMPRD', 'GEOPSR', 'GHGSSC', 'MPLODG', 'PROD', 'RESERV', 'SSFINL', 'SSOTHR', 'SSREPR', 'SURR', 'TRANS', 'WATOTH', 'WATPRR', 'WELAB', 'WELOTH']
for t in types:
    view = df[df.RTYPE == t]
    print(t, view.shape)

CDIFIN (37, 12)
CUMPRD (65, 12)
GEOPSR (1, 12)
GHGSSC (14, 12)
MPLODG (0, 12)
PROD (610, 12)
RESERV (741, 12)
SSFINL (805, 12)
SSOTHR (413, 12)
SSREPR (92, 12)
SURR (2, 12)
TRANS (231, 12)
WATOTH (0, 12)
WATPRR (0, 12)
WELAB (843, 12)
WELOTH (504, 12)


In [71]:
df = df[df.SUBMITBY.str.contains('BHP')==True]

In [72]:
df = df[df.RSTATUS.str.contains('C')==False]  # confidential files will not be available to download

In [88]:
df

,REPNO,RTITLE,RTYPE,RSTATUS,AUTHOR,LODGER,SUBMITBY,LOCALITY,MAPREF,TENURE,REPDATE,RECDATE
9232,27264,"EPC 463, BATHURST RANGE, REPORT FOR THE SIX MO...",6MTH,O,"KITCH, R B",NaN,BHP AUSTRALIA COAL PTY LTD,NW OF COOKTOWN,"7768, 7769",EPC 463,1995-06-01,2006-08-03
9546,23185,"EPM 6031, GYMPIE, SIX MONTHLY REPORT FOR PERIO...",6MTH,O,"ROWLANDS, N",NaN,BHP GOLD MINES LIMITED,GYMPIE AREA,"9346, 9345, 9445",EPM 6031,1990-10-01,2003-11-28
9614,22691,"EPM 6988, AMAROO, REPORT FOR THE SIX MONTHS EN...",6MTH,O,"SMIT, R",NaN,BHP GOLD MINES LIMITED,NNW OF CLERMONT,"8353, 8354",EPM 6988,1990-11-01,2003-11-28
9627,22587,"EPM 6987 (DIAMOND CREEK), 6990 (DOORUNA), 7421...",6MTH,O,"SMIT, R",NaN,BHP MINERALS LIMITED,S OF MOUNT COOLON,"8354, 8353","EPM 6987, EPM 6990, EPM 7421",1990-11-01,2006-03-14
9637,22546,"EPM 6869 (OAKLANDS), 6903 (CEMENT MILLS), REPO...",6MTH,O,"SMIT, R",NaN,BHP MINERALS LIMITED,W OF WARWICK,9241,"EPM 6869, EPM 6903",1990-10-01,2006-04-13
...,...,...,...,...,...,...,...,...,...,...,...,...
96225,22795,"EPM 5899 (STOCKMAN CREEK), 6050 (EINASLEIGH), ...",ANNUAL,O,"SHEPPARD, M J,KENNEDY, R J",NaN,BHP MINERALS LIMITED,EINASLEIGH AREA,"7660, 7760, 7761","EPM 5899, EPM 6050",1991-03-01,2003-08-29
96232,22791,"EPC 463, BATHURST RANGE, ANNUAL REPORT FOR YEA...",ANNUAL,O,"HANDKE, R A",NaN,BHP MINERALS LIMITED,NW OF COOKTOWN,7769,EPC 463,1991-01-01,2006-06-26
96266,22701,"EPM 6959, 7120, KEPPEL SANDS, ANNUAL REPORT FO...",ANNUAL,O,"BENN, C J",NaN,BHP MINERALS LIMITED,E OF ROCKHAMPTON,"9050, 9051","EPM 6959, EPM 7120",1991-06-01,2004-12-17
96405,22003,"A-P 3CR, BATHURST RANGE, ANNUAL REPORT FOR YEA...",ANNUAL,O,"KITCH, R",NaN,BHP MINERALS LIMITED,NW OF COOKTOWN,"7768, 7769",EPC 463,1990-02-01,2006-06-30


In [68]:
df.dtypes

REPNO                int32
RTITLE              object
RTYPE               object
RSTATUS             object
AUTHOR              object
LODGER              object
SUBMITBY            object
LOCALITY            object
MAPREF              object
TENURE              object
REPDATE     datetime64[ns]
RECDATE     datetime64[ns]
dtype: object

In [93]:
from datetime import timedelta, date

def col2datetime(col):  # to stop to_datetime from converting 1900s to 2000s
    df[col] = pd.to_datetime(df[col], dayfirst=True)
    future = df[col] > pd.Timestamp(year=2020,month=1,day=1)
    df.loc[future, col] -= timedelta(days=365.25*100)

In [94]:
df.REPNO = df.REPNO.astype(int)
col2datetime('REPDATE')
col2datetime('RECDATE')

In [87]:
cutoffdate = pd.Timestamp(1990, 1, 1)
mask = df.REPDATE > cutoffdate
df = df[mask]

In [81]:
import random

rs = random.sample(list(df.REPNO), 10)
rs

[29762, 57315, 27958, 56552, 24526, 25755, 54191, 26526, 21895, 43458]

In [82]:
download_reports(rs, 'coal_sample/')

In [89]:
df.to_excel("QDEX_reports_BHP.xlsx")

In [ ]:
import img2pdf 

